# CS 4644: Final Project Models

Copyright (c) 2025 Ethan Nguyen-Tu

## Part 1: Setup

##### STEP 1: Mount Google Drive and Set Device for Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
drive_path = "drive/MyDrive" # NOTE: Separated so that colab can access the '.kaggle' folder in your Google Drive for Kaggle API authentication
project_folder = drive_path + "/CS4644_FinalProject"

##### STEP 2: All Imports

In [3]:
import numpy as np
import pandas as pd
import os
import zipfile
from tqdm import tqdm
from torchvision import datasets, transforms, models
from PIL import Image

In [4]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [5]:
from keras import layers
from keras import models
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

##### STEP 3: Seed for Reproducability

In [6]:
SEED = 8

np.random.seed(SEED)
torch.manual_seed(SEED)
tf.random.set_seed(SEED)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(SEED)

##### STEP 4: General Helper Functions

In [ ]:
def train_torch(model, optimizer=None, criterion=nn.CrossEntropyLoss(), learning_rate=0.0001):
  model.to(DEVICE)

  if not optimizer:
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  train_losses = []
  train_accuracies = []

  val_losses = []
  val_accuracies = []

  for epoch in range(EPOCHS):

      model.train()

      train_correct = 0
      train_total = 0
      train_running_loss = 0.0

      for imgs, labels in train_loader:

          # STEP: Send tensors to device
          imgs = imgs.to(DEVICE)
          labels = labels.long().to(DEVICE)

          optimizer.zero_grad()

          outputs = model(imgs)
          loss = criterion(outputs, labels)

          loss.backward()
          optimizer.step()

          # STEP: Calculate training loss
          train_running_loss += loss.item()

          # STEP: Calculate training accuracy
          _, predictions = torch.max(outputs.data, 1)
          train_correct += (predictions == labels).sum().item()
          train_total += labels.size(0)

      # STEP: Calculate average training metrics
      avg_train_loss = train_running_loss / len(train_loader)
      train_losses.append(avg_train_loss)

      avg_train_acc = train_correct / train_total
      train_accuracies.append(avg_train_acc)


      model.eval()

      val_correct = 0
      val_total = 0
      val_running_loss = 0.0

      with torch.no_grad():
          for imgs, labels in val_loader:
              # STEP: Send tensors to device
              imgs = imgs.to(DEVICE)
              labels = labels.long().to(DEVICE)

              outputs = model(imgs)

              # STEP: Calculate validation loss
              val_loss = criterion(outputs, labels)
              val_running_loss += val_loss.item()

              # STEP: Calculate validation accuracy
              _, predictions = torch.max(outputs.data, 1)
              val_correct += (predictions == labels).sum().item()
              val_total += labels.size(0)

      # STEP: Calculate average validation metrics
      avg_val_loss = val_running_loss / len(val_loader)
      val_losses.append(avg_val_loss)

      avg_val_acc = val_correct / val_total
      val_accuracies.append(avg_val_acc)


      # STEP: Output Progress
      print(f"Epoch {epoch+1}/{EPOCHS} — accuracy: {avg_train_acc:.4f} - loss: {avg_train_loss:.4f} - val_accuracy {avg_val_acc:.4f} - val_loss: {avg_val_loss:.4f}")

  return pd.DataFrame({
      'accuracy': train_accuracies,
      'loss': train_losses,
      'val_accuracy': val_accuracies,
      'val_loss': val_losses
  })

In [35]:
def test_pytorch(model, criterion=nn.CrossEntropyLoss()):
  model.eval()

  test_correct = 0
  test_total = 0
  test_running_loss = 0.0

  with torch.no_grad():
      for imgs, labels in test_loader:
          # STEP: Send tensors to device
          imgs = imgs.to(DEVICE)
          labels = labels.long().to(DEVICE)

          outputs = model(imgs)

          # STEP: Calculate validation loss
          test_loss = criterion(outputs, labels)
          test_running_loss += test_loss.item()

          # STEP: Calculate validation accuracy
          _, predictions = torch.max(outputs.data, 1)
          test_correct += (predictions == labels).sum().item()
          test_total += labels.size(0)

  # STEP: Calculate average validation metrics
  avg_test_loss = test_running_loss / len(test_loader)
  avg_test_acc = test_correct / test_total

  print(f"Test accuracy: {avg_test_acc:.4f} - Test loss: {avg_test_loss:.4f}")

## PART 2: Load the Datasets

In [8]:
def zip_to_colab(zip_file_path, extract_dir_name):
  extract_dir = '/content/' + extract_dir_name + "/"
  os.makedirs(extract_dir, exist_ok=True)
  before_file_count = len(os.listdir(extract_dir))

  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extract_dir)

  files_extracted = len(os.listdir(extract_dir)) - before_file_count
  print(f"Files from {zip_file_path} extracted to: {extract_dir}")
  print("Number of files extracted:", files_extracted)
  return files_extracted

##### STEP 1: Load the "Human Faces" Dataset

Dataset Source: ["Human Faces" by Ashwin Gupta](https://www.kaggle.com/datasets/ashwingupta3012/human-faces/data)

In [ ]:
# zip_to_colab(project_folder + '/HumanFacesCleaned.zip', "HumanFacesImages")
# HUMANFACES_IMAGE_EXTENSIONS = {'.png', '.JPG', '.jpeg', '.jpg'}

##### STEP 2: Load the "Fake-Vs-Real-Faces (Hard)" Dataset

Dataset Source: ["Fake-Vs-Real-Faces (Hard)" by Hamza Boulahi](https://www.kaggle.com/datasets/hamzaboulahia/hardfakevsrealfaces)

In [ ]:
file_count = zip_to_colab(project_folder + '/RealImages.zip', 'FakeVsRealFacesImages')
file_count += zip_to_colab(project_folder + '/FakeImages.zip', 'FakeVsRealFacesImages')
assert len(os.listdir('/content/FakeVsRealFacesImages/')) == file_count # Check to make sure no files were overridden

Files from drive/MyDrive/CS4644_FinalProject/RealImages.zip extracted to: /content/FakeVsRealFacesImages/
Number of files extracted: 589
Files from drive/MyDrive/CS4644_FinalProject/FakeImages.zip extracted to: /content/FakeVsRealFacesImages/
Number of files extracted: 700


In [ ]:
FAKEVREAL_IMAGE_EXTENSIONS = {'.jpg'}
FAKEVREAL_IMAGE_PATH = '/content/FakeVsRealFacesImages/'

##### STEP 3: Load the "deepfake and real images" Dataset

Datset Source: ["deepfake and real images" by Manjil Karki](https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images)

In [9]:
file_count = zip_to_colab(project_folder + '/Test_RealImages.zip', 'TestDataset')
file_count += zip_to_colab(project_folder + '/Test_FakeImages.zip', 'TestDataset')
assert len(os.listdir('/content/TestDataset/')) == file_count # Check to make sure no files were overridden

Files from drive/MyDrive/CS4644_FinalProject/Test_RealImages.zip extracted to: /content/TestDataset/
Number of files extracted: 129
Files from drive/MyDrive/CS4644_FinalProject/Test_FakeImages.zip extracted to: /content/TestDataset/
Number of files extracted: 129


In [17]:
TEST_IMAGE_EXTENSIONS = {'.jpg'}
TEST_IMAGE_PATH = '/content/TestDataset/'

## PART 3: Get Test, Train, and Validation Sets

In [11]:
def inspect_df(dataframe, rows=10):
  count_real = dataframe[dataframe['label'] == 1].shape[0]
  count_fake = dataframe[dataframe['label'] == 0].shape[0]
  assert count_real + count_fake == dataframe.shape[0]
  print("# of Real Images:", count_real)
  print("# of Fake Images:", count_fake)
  print("% Real:", (100 * count_real) // dataframe.shape[0])
  print("% Fake:", (100 * count_fake) // dataframe.shape[0])
  return dataframe.sample(frac=1, random_state=SEED).head(rows)

Step 1: Load the dataset labels into a dataframe.

In [12]:
df_TestDataset = pd.read_csv(project_folder + "/test_data.csv")
df_TestDataset.head(10)

,images_id,label
0,fake_5234.jpg,0
1,fake_366.jpg,0
2,fake_4479.jpg,0
3,fake_2599.jpg,0
4,fake_577.jpg,0
5,fake_2475.jpg,0
6,fake_2431.jpg,0
7,fake_3544.jpg,0
8,fake_1061.jpg,0
9,fake_200.jpg,0


In [ ]:
df_RealVFake = pd.read_csv(project_folder + "/RealVFakeLabels.csv")
df_RealVFake.head(10)

,images_id,label
0,real_1,real
1,real_10,real
2,real_100,real
3,real_101,real
4,real_102,real
5,real_103,real
6,real_104,real
7,real_105,real
8,real_106,real
9,real_107,real


STEP 2: Replace the 'real' and 'fake' labelings with 1 and 0 respectively if needed.

In [ ]:
df_RealVFake.loc[df_RealVFake['label'] == 'real', 'label'] = 1
df_RealVFake.loc[df_RealVFake['label'] == 'fake', 'label'] = 0
inspect_df(df_RealVFake)

# of Real Images: 589
# of Fake Images: 700
% Real: 45
% Fake: 54


,images_id,label
1123,fake_580,0
1265,fake_78,0
385,real_446,1
955,fake_429,0
437,real_493,1
230,real_306,1
1107,fake_566,0
688,fake_189,0
111,real_2,1
223,real_30,1


STEP 3: Randomly separate the rows into an 80% train set and a 20% validation set.

In [ ]:
# Shuffle Dataset
df_RealVFake_shuffled = df_RealVFake.sample(frac=1, random_state=SEED)

# Calculate the split index
separation_idx = int(0.8 * len(df_RealVFake_shuffled))

# Split data into train and test sets
df_train_RealVFake = df_RealVFake_shuffled[:separation_idx]
df_test_RealVFake = df_RealVFake_shuffled[separation_idx:]

print("Train set shape:", df_train_RealVFake.shape)
print("Validation set shape:", df_test_RealVFake.shape)

Train set shape: (1031, 2)
Test set shape: (258, 2)


In [ ]:
inspect_df(df_train_RealVFake)

# of Real Images: 470
# of Fake Images: 561
% Real: 45
% Fake: 54


,images_id,label
1097,fake_557,0
225,real_301,1
1260,fake_73,0
857,fake_340,0
468,real_520,1
1095,fake_555,0
232,real_308,1
292,real_362,1
1104,fake_563,0
640,fake_145,0


In [ ]:
inspect_df(df_test_RealVFake)

# of Real Images: 119
# of Fake Images: 139
% Real: 46
% Fake: 53


,images_id,label
178,real_26,1
528,real_575,1
480,real_531,1
1105,fake_564,0
168,real_250,1
1167,fake_62,0
136,real_221,1
142,real_227,1
42,real_137,1
1038,fake_503,0


In [13]:
inspect_df(df_TestDataset)

# of Real Images: 129
# of Fake Images: 129
% Real: 50
% Fake: 50


,images_id,label
106,fake_4118.jpg,0
193,real_321.jpg,1
100,fake_4438.jpg,0
227,real_2071.jpg,1
30,fake_3354.jpg,0
25,fake_5083.jpg,0
253,real_4180.jpg,1
79,fake_3526.jpg,0
77,fake_4860.jpg,0
146,real_1616.jpg,1


## PART 4: Base Models

##### Reference Paper Keras Model

Original Paper Reference: https://philarchive.org/archive/SALCOR-3

In [ ]:
def get_keras_ref_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu',
    input_shape=(256, 256, 3)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(256, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(512, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dense(2, activation='softmax')) # Changed to 2 from 4 in order to fit target shape

  return model

##### PyTorch Conversion

Convert the Reference Paper Keras Model to PyTorch

In [32]:
# The Reference Model Converted

def get_pytorch_ref_model():
  conv2d_ks = 3 # Conv2d Kernel Size
  conv2d_pad = 1 # Conv2d Padding
  conv2d_s = 1 # Conv2d Stride

  maxpool2d_ks = 2 # MaxPool2d Kernel Size
  maxpool2d_s = 2 # MaxPool2d Stride

  return nn.Sequential(
      nn.Conv2d(3, 32, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(32, 64, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(64, 128, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(128, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(256, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Conv2d(256, 512, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
      nn.Flatten(),
      nn.Linear(8192, 512),
      nn.ReLU(),
      nn.Linear(512, 2), # Changed to 2 from 4 in order to fit target shape
      # nn.Softmax(), # Removed since loss applies sigmoid function internally
  )

## PART 5: Establishing the Baseline

Goal: Confirm that the Keras Model and PyTorch Conversion have the same accuracy on both the test and train set.

In [14]:
IMG_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.0001 # Pg. 8 of "Classification of Real and Fake Human Faces Using Deep Learning" by Fatima Maher Salman and Samy S. Abu-Naser
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
def basic_process_data(dataset, path, extension='.jpg'):
    keras_data = []
    torch_data = []
    labels = []

    for _, row in dataset.iterrows():
        image_path = ''.join([path, row["images_id"], extension])

        try:
            img = Image.open(image_path).resize((IMG_SIZE,IMG_SIZE))
            img = np.array(img) / float(IMG_SIZE)
            keras_data.append(img)
            img = torch.FloatTensor(img).permute(2, 0, 1)
            torch_data.append(img)
            labels.append(row["label"])

        except Exception as e:
            print(f"Error loading or processing image {image_path}: {e}")

    num_classes = len(set(labels))
    torch_labels = torch.tensor(labels, dtype=torch.float32).squeeze(0).to(DEVICE)
    torch_data = torch.stack(torch_data).to(DEVICE)

    return np.array(keras_data), to_categorical(np.array(labels), num_classes=num_classes), torch_data, torch_labels

In [ ]:
fakeVReal_TrainData, fakeVReal_TrainLabels, fakeVReal_TrainData_T, fakeVReal_TrainLabels_T = basic_process_data(df_train_RealVFake, FAKEVREAL_IMAGE_PATH)
fakeVReal_TestData, fakeVReal_TestLabels, fakeVReal_TestData_T, fakeVReal_TestLabels_T = basic_process_data(df_test_RealVFake, FAKEVREAL_IMAGE_PATH)

In [22]:
GENERATOR = torch.Generator()
GENERATOR.manual_seed(SEED)

##### STEP 1: Train the Base Keras Model

In [ ]:
model_reference = get_keras_ref_model()
model_reference.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

model_reference_results = model_reference.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_TestData, fakeVReal_TestLabels))
model_reference_history = pd.DataFrame(model_reference_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 241ms/step - accuracy: 0.5612 - loss: 0.6757 - val_accuracy: 0.7248 - val_loss: 0.6041
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 192ms/step - accuracy: 0.7921 - loss: 0.5608 - val_accuracy: 0.8953 - val_loss: 0.3419
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 194ms/step - accuracy: 0.9288 - loss: 0.2594 - val_accuracy: 0.9729 - val_loss: 0.0755
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 205ms/step - accuracy: 0.9800 - loss: 0.0632 - val_accuracy: 0.9884 - val_loss: 0.0239
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - accuracy: 0.9900 - loss: 0.0292 - val_accuracy: 0.9961 - val_loss: 0.0153
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 218ms/step - accuracy: 0.9998 - loss: 0.0083 - val_accuracy: 0.9884 - val_loss: 0.0212
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accuracy: 0.9922 - val_loss: 0.0187
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - accuracy: 1.0000 - loss: 0.0042 - val_accuracy: 0.

In [ ]:
model_reference_results.history

{'accuracy': [0.5935984253883362,
  0.8128030896186829,
  0.9495635032653809,
  0.9786614775657654,
  0.9912706017494202,
  0.9990300536155701,
  1.0,
  1.0,
  1.0,
  1.0],
 'loss': [0.6607054471969604,
  0.5154953002929688,
  0.19862763583660126,
  0.05644642934203148,
  0.02531432919204235,
  0.007643189746886492,
  0.005164767149835825,
  0.003601004835218191,
  0.0020356669556349516,
  0.0011404079850763083],
 'val_accuracy': [0.7248061895370483,
  0.895348846912384,
  0.9728682041168213,
  0.9883720874786377,
  0.9961240291595459,
  0.9883720874786377,
  0.9922480583190918,
  0.9961240291595459,
  0.9961240291595459,
  0.9961240291595459],
 'val_loss': [0.6040947437286377,
  0.34191980957984924,
  0.07550416141748428,
  0.023859750479459763,
  0.015338250435888767,
  0.02123924158513546,
  0.018742287531495094,
  0.014542038552463055,
  0.01286748144775629,
  0.01786462962627411]}

In [ ]:
model_reference_history.to_csv(''.join(['/content/', project_folder, "/History/model_reference_history.csv"]))
model_reference_history

,accuracy,loss,val_accuracy,val_loss
0,0.593598,0.660705,0.724806,0.604095
1,0.812803,0.515495,0.895349,0.341920
2,0.949564,0.198628,0.972868,0.075504
3,0.978661,0.056446,0.988372,0.023860
4,0.991271,0.025314,0.996124,0.015338
5,0.999030,0.007643,0.988372,0.021239
6,1.000000,0.005165,0.992248,0.018742
7,1.000000,0.003601,0.996124,0.014542
8,1.000000,0.002036,0.996124,0.012867
9,1.000000,0.001140,0.996124,0.017865


In [ ]:
model_reference.save(''.join(['/content/', project_folder, "/Models/model_reference.keras"]))

##### STEP 2: Train the Base PyTorch Conversion Model

In [ ]:
train_dataset = torch.utils.data.TensorDataset(fakeVReal_TrainData_T, fakeVReal_TrainLabels_T)
test_dataset = torch.utils.data.TensorDataset(fakeVReal_TestData_T, fakeVReal_TestLabels_T)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=GENERATOR)
val_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model_pytorch_conversion = get_pytorch_ref_model()
pytorch_conversion_history = train_torch(model_pytorch_conversion, learning_rate=LEARNING_RATE)

Epoch 1/10 — accuracy: 0.5403 - loss: 0.6895 - val_accuracy 0.5388 - val_loss: 0.6994
Epoch 2/10 — accuracy: 0.5441 - loss: 0.6769 - val_accuracy 0.6240 - val_loss: 0.6677
Epoch 3/10 — accuracy: 0.7808 - loss: 0.5071 - val_accuracy 0.8760 - val_loss: 0.2930
Epoch 4/10 — accuracy: 0.8962 - loss: 0.2345 - val_accuracy 0.9535 - val_loss: 0.0990
Epoch 5/10 — accuracy: 0.9554 - loss: 0.1253 - val_accuracy 0.9690 - val_loss: 0.0706
Epoch 6/10 — accuracy: 0.9680 - loss: 0.0884 - val_accuracy 0.9884 - val_loss: 0.0402
Epoch 7/10 — accuracy: 0.9835 - loss: 0.0530 - val_accuracy 0.9845 - val_loss: 0.0352
Epoch 8/10 — accuracy: 0.9855 - loss: 0.0360 - val_accuracy 0.9961 - val_loss: 0.0310
Epoch 9/10 — accuracy: 0.9942 - loss: 0.0246 - val_accuracy 0.9961 - val_loss: 0.0323
Epoch 10/10 — accuracy: 0.9952 - loss: 0.0153 - val_accuracy 0.9845 - val_loss: 0.0345


In [ ]:
pytorch_conversion_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_conversion_history.csv"]))
pytorch_conversion_history

,accuracy,loss,val_accuracy,val_loss
0,0.540252,0.689503,0.538760,0.699417
1,0.544132,0.676888,0.624031,0.667686
2,0.780795,0.507133,0.875969,0.293042
3,0.896217,0.234463,0.953488,0.099015
4,0.955383,0.125338,0.968992,0.070615
5,0.967992,0.088435,0.988372,0.040238
6,0.983511,0.053033,0.984496,0.035191
7,0.985451,0.036019,0.996124,0.030963
8,0.994180,0.024632,0.996124,0.032331
9,0.995150,0.015329,0.984496,0.034484


In [ ]:
torch.save(model_pytorch_conversion.state_dict(), ''.join(['/content/', project_folder, "/Models/pytorch_conversion.pth"]))

## PART 6: Optimized Models

#### Optimization - Learning Rate

The Keras Reference Model / PyTorch Conversion trained with 10x the learning rate of the baseline and 100x the learning rate of the baseline.

##### 1. Setup

In [ ]:
LEARNING_RATE_10x = 10 * LEARNING_RATE
LEARNING_RATE_100x = 100 * LEARNING_RATE

##### 2. Keras Model

###### 10x LR Keras

In [ ]:
keras_10x_LR = get_keras_ref_model()
keras_10x_LR.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_10x),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_10x_LR_results = keras_10x_LR.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_TestData, fakeVReal_TestLabels))
keras_10x_LR_history = pd.DataFrame(keras_10x_LR_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step - accuracy: 0.5046 - loss: 0.7276 - val_accuracy: 0.5388 - val_loss: 0.6849
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 191ms/step - accuracy: 0.5731 - loss: 0.6678 - val_accuracy: 0.5930 - val_loss: 0.5816
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - accuracy: 0.8462 - loss: 0.4066 - val_accuracy: 0.9729 - val_loss: 0.0860
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - accuracy: 0.9846 - loss: 0.0612 - val_accuracy: 0.9884 - val_loss: 0.0760
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 193ms/step - accuracy: 0.9939 - loss: 0.0191 - val_accuracy: 0.9729 - val_loss: 0.0960
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 205ms/step - accuracy: 0.9933 - loss: 0.0168 - val_accuracy: 0.9922 - val_loss: 0.0542
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 199ms/step - accuracy: 0.9925 - loss: 0.0166 - val_accuracy: 0.9884 - val_loss: 0.0510
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 223ms/step - accuracy: 0.9990 - loss: 0.0035 - val_accuracy: 0.

In [ ]:
keras_10x_LR_results.history

{'accuracy': [0.5140640139579773,
  0.6207565665245056,
  0.9146459698677063,
  0.9873908758163452,
  0.99321049451828,
  0.9951503276824951,
  0.9912706017494202,
  0.9980601072311401,
  1.0,
  1.0],
 'loss': [0.7093977928161621,
  0.6726914048194885,
  0.2823115885257721,
  0.053578708320856094,
  0.023343829438090324,
  0.02092837169766426,
  0.02133243903517723,
  0.00762065127491951,
  0.0017939809476956725,
  0.00028033758280798793],
 'val_accuracy': [0.538759708404541,
  0.5930232405662537,
  0.9728682041168213,
  0.9883720874786377,
  0.9728682041168213,
  0.9922480583190918,
  0.9883720874786377,
  0.9806201457977295,
  0.9844961166381836,
  0.9844961166381836],
 'val_loss': [0.6849468350410461,
  0.5815573334693909,
  0.08596669882535934,
  0.07596595585346222,
  0.09595980495214462,
  0.05417780205607414,
  0.051000162959098816,
  0.042811695486307144,
  0.05967644602060318,
  0.08886647969484329]}

In [ ]:
keras_10x_LR_history.to_csv(''.join(['/content/', project_folder, "/History/keras_10x_LR_history.csv"]))
keras_10x_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.514064,0.709398,0.538760,0.684947
1,0.620757,0.672691,0.593023,0.581557
2,0.914646,0.282312,0.972868,0.085967
3,0.987391,0.053579,0.988372,0.075966
4,0.993210,0.023344,0.972868,0.095960
5,0.995150,0.020928,0.992248,0.054178
6,0.991271,0.021332,0.988372,0.051000
7,0.998060,0.007621,0.980620,0.042812
8,1.000000,0.001794,0.984496,0.059676
9,1.000000,0.000280,0.984496,0.088866


In [ ]:
keras_10x_LR.save(''.join(['/content/', project_folder, "/Models/keras_10x_LR.keras"]))

###### 100x LR Keras

In [ ]:
keras_100x_LR = get_keras_ref_model()
keras_100x_LR.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_100x),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_100x_LR_results = keras_100x_LR.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_TestData, fakeVReal_TestLabels))
keras_100x_LR_history = pd.DataFrame(keras_100x_LR_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 11s 292ms/step - accuracy: 0.4895 - loss: 29.4107 - val_accuracy: 0.5465 - val_loss: 0.6902
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 201ms/step - accuracy: 0.5058 - loss: 9.1945 - val_accuracy: 0.5388 - val_loss: 0.6902
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 213ms/step - accuracy: 0.5256 - loss: 0.6928 - val_accuracy: 0.5388 - val_loss: 0.6970
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 201ms/step - accuracy: 0.5029 - loss: 0.6961 - val_accuracy: 0.5388 - val_loss: 0.6931
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 183ms/step - accuracy: 0.5660 - loss: 0.6839 - val_accuracy: 0.4457 - val_loss: 1.2658
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 200ms/step - accuracy: 0.5263 - loss: 0.9090 - val_accuracy: 0.5388 - val_loss: 0.6911
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 202ms/step - accuracy: 0.4917 - loss: 0.7020 - val_accuracy: 0.5388 - val_loss: 0.6903
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.5326 - loss: 0.6918 - val_accuracy: 

In [ ]:
keras_100x_LR_results.history

{'accuracy': [0.48690590262413025,
  0.5150339603424072,
  0.5218234658241272,
  0.5237633585929871,
  0.5848690867424011,
  0.534432590007782,
  0.5189136862754822,
  0.5441319346427917,
  0.5441319346427917,
  0.5441319346427917],
 'loss': [15.791860580444336,
  5.8137078285217285,
  0.6923055052757263,
  0.6909670829772949,
  0.6765652894973755,
  0.790690004825592,
  0.6964243054389954,
  0.689582884311676,
  0.6910028457641602,
  0.6902890801429749],
 'val_accuracy': [0.5465116500854492,
  0.538759708404541,
  0.538759708404541,
  0.538759708404541,
  0.44573643803596497,
  0.538759708404541,
  0.538759708404541,
  0.538759708404541,
  0.538759708404541,
  0.538759708404541],
 'val_loss': [0.690218985080719,
  0.6902278065681458,
  0.697045624256134,
  0.6930888891220093,
  1.2658246755599976,
  0.6911484003067017,
  0.6903395652770996,
  0.691942036151886,
  0.6903076767921448,
  0.6901516318321228]}

In [ ]:
keras_100x_LR_history.to_csv(''.join(['/content/', project_folder, "/History/keras_100x_LR_history.csv"]))
keras_100x_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.486906,15.791861,0.546512,0.690219
1,0.515034,5.813708,0.538760,0.690228
2,0.521823,0.692306,0.538760,0.697046
3,0.523763,0.690967,0.538760,0.693089
4,0.584869,0.676565,0.445736,1.265825
5,0.534433,0.790690,0.538760,0.691148
6,0.518914,0.696424,0.538760,0.690340
7,0.544132,0.689583,0.538760,0.691942
8,0.544132,0.691003,0.538760,0.690308
9,0.544132,0.690289,0.538760,0.690152


In [ ]:
keras_100x_LR.save(''.join(['/content/', project_folder, "/Models/keras_100x_LR.keras"]))

##### 3. PyTorch Model

###### 10x LR PyTorch

In [ ]:
pyTorch_10x_LR = get_pytorch_ref_model()
pytorch_10X_LR_history = train_torch(pyTorch_10x_LR, learning_rate=LEARNING_RATE_10x)

Epoch 1/10 — accuracy: 0.5286 - loss: 0.6964 - val_accuracy 0.5388 - val_loss: 0.7030
Epoch 2/10 — accuracy: 0.5441 - loss: 0.6875 - val_accuracy 0.5388 - val_loss: 0.7036
Epoch 3/10 — accuracy: 0.7051 - loss: 0.5424 - val_accuracy 0.8682 - val_loss: 0.3008
Epoch 4/10 — accuracy: 0.9302 - loss: 0.1802 - val_accuracy 0.9264 - val_loss: 0.1579
Epoch 5/10 — accuracy: 0.9602 - loss: 0.0918 - val_accuracy 0.9767 - val_loss: 0.0569
Epoch 6/10 — accuracy: 0.9913 - loss: 0.0270 - val_accuracy 0.9884 - val_loss: 0.0407
Epoch 7/10 — accuracy: 0.9981 - loss: 0.0063 - val_accuracy 0.9922 - val_loss: 0.0462
Epoch 8/10 — accuracy: 0.9990 - loss: 0.0021 - val_accuracy 0.9767 - val_loss: 0.0471
Epoch 9/10 — accuracy: 0.9952 - loss: 0.0158 - val_accuracy 0.9961 - val_loss: 0.0121
Epoch 10/10 — accuracy: 0.9971 - loss: 0.0103 - val_accuracy 0.9884 - val_loss: 0.0552


In [ ]:
pytorch_10X_LR_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_10X_LR_history.csv"]))
pytorch_10X_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.528613,0.696424,0.538760,0.702956
1,0.544132,0.687492,0.538760,0.703638
2,0.705141,0.542407,0.868217,0.300824
3,0.930165,0.180225,0.926357,0.157938
4,0.960233,0.091826,0.976744,0.056932
5,0.991271,0.026984,0.988372,0.040748
6,0.998060,0.006330,0.992248,0.046156
7,0.999030,0.002072,0.976744,0.047103
8,0.995150,0.015752,0.996124,0.012086
9,0.997090,0.010268,0.988372,0.055203


In [ ]:
torch.save(pyTorch_10x_LR.state_dict(), ''.join(['/content/', project_folder, "/Models/pytorch_10X_LR.pth"]))

###### 100x LR PyTorch

In [ ]:
pyTorch_100x_LR = get_pytorch_ref_model()
pytorch_100X_LR_history = train_torch(pyTorch_100x_LR, learning_rate=LEARNING_RATE_100x)

Epoch 1/10 — accuracy: 0.4811 - loss: 32.2185 - val_accuracy 0.4612 - val_loss: 0.6887
Epoch 2/10 — accuracy: 0.5131 - loss: 1.4735 - val_accuracy 0.5388 - val_loss: 0.7053
Epoch 3/10 — accuracy: 0.5441 - loss: 0.7513 - val_accuracy 0.5388 - val_loss: 0.7291
Epoch 4/10 — accuracy: 0.5432 - loss: 0.6912 - val_accuracy 0.5388 - val_loss: 0.6992
Epoch 5/10 — accuracy: 0.5441 - loss: 0.6894 - val_accuracy 0.5388 - val_loss: 0.7170
Epoch 6/10 — accuracy: 0.5441 - loss: 0.6924 - val_accuracy 0.5388 - val_loss: 0.7100
Epoch 7/10 — accuracy: 0.5441 - loss: 0.6905 - val_accuracy 0.5388 - val_loss: 0.7011
Epoch 8/10 — accuracy: 0.5441 - loss: 0.6893 - val_accuracy 0.5388 - val_loss: 0.7083
Epoch 9/10 — accuracy: 0.5441 - loss: 0.6892 - val_accuracy 0.5388 - val_loss: 0.7087
Epoch 10/10 — accuracy: 0.5441 - loss: 0.6877 - val_accuracy 0.5388 - val_loss: 0.7101


In [ ]:
pytorch_100X_LR_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_100X_LR_history.csv"]))
pytorch_100X_LR_history

,accuracy,loss,val_accuracy,val_loss
0,0.481086,32.218477,0.46124,0.688694
1,0.513094,1.473510,0.53876,0.705333
2,0.544132,0.751336,0.53876,0.729134
3,0.543162,0.691236,0.53876,0.699245
4,0.544132,0.689422,0.53876,0.716976
5,0.544132,0.692355,0.53876,0.710030
6,0.544132,0.690466,0.53876,0.701136
7,0.544132,0.689263,0.53876,0.708256
8,0.544132,0.689237,0.53876,0.708749
9,0.544132,0.687749,0.53876,0.710134


In [ ]:
torch.save(pyTorch_100x_LR.state_dict(), ''.join(['/content/', project_folder, "/Models/pytorch_100X_LR.pth"]))

#### Optimization - Extra Layer

The Keras Reference Model / PyTorch Conversion with an additional convolution layer followed by relu and maxpool2d.

##### 1. Keras Model

In [ ]:
keras_additional_conv = models.Sequential()
keras_additional_conv.add(layers.Conv2D(32, (3, 3), activation='relu',
  input_shape=(256, 256, 3)))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(64, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(128, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(256, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(256, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(512, (3, 3), activation='relu'))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Conv2D(1024, (3, 3), activation='relu', padding="same"))
keras_additional_conv.add(layers.MaxPooling2D((2, 2)))
keras_additional_conv.add(layers.Flatten())
keras_additional_conv.add(layers.Dense(1024, activation='relu'))
keras_additional_conv.add(layers.Dense(2, activation='softmax')) # Changed to 2 from 4 in order to fit target shape

In [ ]:
keras_additional_conv.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_additional_conv_results = keras_additional_conv.fit(fakeVReal_TrainData, fakeVReal_TrainLabels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_TestData, fakeVReal_TestLabels))
keras_additional_conv_history = pd.DataFrame(keras_additional_conv_results.history)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 9s 249ms/step - accuracy: 0.5513 - loss: 0.6828 - val_accuracy: 0.6744 - val_loss: 0.6289
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 188ms/step - accuracy: 0.7430 - loss: 0.5876 - val_accuracy: 0.8953 - val_loss: 0.3474
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 192ms/step - accuracy: 0.9186 - loss: 0.2558 - val_accuracy: 0.9690 - val_loss: 0.0857
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 189ms/step - accuracy: 0.9787 - loss: 0.0658 - val_accuracy: 0.9690 - val_loss: 0.0772
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 189ms/step - accuracy: 0.9740 - loss: 0.0580 - val_accuracy: 0.9845 - val_loss: 0.0381
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 199ms/step - accuracy: 0.9854 - loss: 0.0455 - val_accuracy: 0.9961 - val_loss: 0.0233
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 195ms/step - accuracy: 0.9992 - loss: 0.0135 - val_accuracy: 0.9922 - val_loss: 0.0225
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 208ms/step - accuracy: 1.0000 - loss: 0.0033 - val_accuracy: 0.

In [ ]:
keras_additional_conv_results.history

{'accuracy': [0.5771095752716064,
  0.7643064856529236,
  0.934044599533081,
  0.9815713167190552,
  0.9747817516326904,
  0.9912706017494202,
  0.9980601072311401,
  1.0,
  1.0,
  1.0],
 'loss': [0.6730430126190186,
  0.5426245927810669,
  0.19870033860206604,
  0.0521104522049427,
  0.057370997965335846,
  0.03971492871642113,
  0.011663800105452538,
  0.0035857355687767267,
  0.0011300754267722368,
  0.0008490405161865056],
 'val_accuracy': [0.6744186282157898,
  0.895348846912384,
  0.9689922332763672,
  0.9689922332763672,
  0.9844961166381836,
  0.9961240291595459,
  0.9922480583190918,
  0.9961240291595459,
  0.9922480583190918,
  0.9961240291595459],
 'val_loss': [0.6288677453994751,
  0.34736406803131104,
  0.08574407547712326,
  0.07716597616672516,
  0.03813686966896057,
  0.023253394290804863,
  0.02253262884914875,
  0.016723591834306717,
  0.020943783223628998,
  0.01920337602496147]}

In [ ]:
keras_additional_conv_history.to_csv(''.join(['/content/', project_folder, "/History/keras_additional_conv_history.csv"]))
keras_additional_conv_history

,accuracy,loss,val_accuracy,val_loss
0,0.577110,0.673043,0.674419,0.628868
1,0.764306,0.542625,0.895349,0.347364
2,0.934045,0.198700,0.968992,0.085744
3,0.981571,0.052110,0.968992,0.077166
4,0.974782,0.057371,0.984496,0.038137
5,0.991271,0.039715,0.996124,0.023253
6,0.998060,0.011664,0.992248,0.022533
7,1.000000,0.003586,0.996124,0.016724
8,1.000000,0.001130,0.992248,0.020944
9,1.000000,0.000849,0.996124,0.019203


In [ ]:
keras_additional_conv.save(''.join(['/content/', project_folder, "/Models/keras_additional_conv.keras"]))

##### 2. PyTorch Model

In [41]:
def get_pytorch_ref_model_more_layers():
  conv2d_ks = 3 # Conv2d Kernel Size
  conv2d_pad = 1 # Conv2d Padding
  conv2d_s = 1 # Conv2d Stride

  maxpool2d_ks = 2 # MaxPool2d Kernel Size
  maxpool2d_s = 2 # MaxPool2d Stride

  return nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(32, 64, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(64, 128, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(128, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(256, 256, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(256, 512, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Conv2d(512, 1024, kernel_size=conv2d_ks, padding=conv2d_pad, stride=conv2d_s),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=maxpool2d_ks, stride=maxpool2d_s),
    nn.Flatten(),
    nn.Linear(4096, 512),
    nn.ReLU(),
    nn.Linear(512, 2), # Changed to 2 from 4 in order to fit target shape
    # nn.Softmax(), # Removed since loss applies sigmoid function internally
  )

In [ ]:
pyTorch_additional_conv = get_pytorch_ref_model_more_layers()
pyTorch_additional_conv_history = train_torch(pyTorch_additional_conv, learning_rate=LEARNING_RATE)

Epoch 1/10 — accuracy: 0.5383 - loss: 0.6906 - val_accuracy 0.5388 - val_loss: 0.7009
Epoch 2/10 — accuracy: 0.5441 - loss: 0.6881 - val_accuracy 0.5388 - val_loss: 0.6969
Epoch 3/10 — accuracy: 0.6382 - loss: 0.6051 - val_accuracy 0.8217 - val_loss: 0.3908
Epoch 4/10 — accuracy: 0.8894 - loss: 0.2792 - val_accuracy 0.9729 - val_loss: 0.0844
Epoch 5/10 — accuracy: 0.9651 - loss: 0.0989 - val_accuracy 0.9922 - val_loss: 0.0323
Epoch 6/10 — accuracy: 0.9864 - loss: 0.0453 - val_accuracy 0.9961 - val_loss: 0.0074
Epoch 7/10 — accuracy: 0.9961 - loss: 0.0149 - val_accuracy 0.9961 - val_loss: 0.0098
Epoch 8/10 — accuracy: 1.0000 - loss: 0.0037 - val_accuracy 0.9961 - val_loss: 0.0065
Epoch 9/10 — accuracy: 1.0000 - loss: 0.0028 - val_accuracy 1.0000 - val_loss: 0.0028
Epoch 10/10 — accuracy: 1.0000 - loss: 0.0012 - val_accuracy 0.9961 - val_loss: 0.0049


In [ ]:
pyTorch_additional_conv_history.to_csv(''.join(['/content/', project_folder, "/History/pytorch_additional_conv_history.csv"]))
pyTorch_additional_conv_history

,accuracy,loss,val_accuracy,val_loss
0,0.538312,0.690637,0.538760,0.700919
1,0.544132,0.688067,0.538760,0.696891
2,0.638215,0.605140,0.821705,0.390768
3,0.889428,0.279198,0.972868,0.084396
4,0.965082,0.098923,0.992248,0.032311
5,0.986421,0.045268,0.996124,0.007440
6,0.996120,0.014885,0.996124,0.009769
7,1.000000,0.003732,0.996124,0.006544
8,1.000000,0.002829,1.000000,0.002805
9,1.000000,0.001208,0.996124,0.004945


In [ ]:
torch.save(pyTorch_additional_conv.state_dict(), ''.join(['/content/', project_folder, "/Models/pyTorch_additional_conv.pth"]))

#### Optimization - More Image Transforms

Same as the reference model, but the images transformations are added to the model's training data.


##### 1. Setup

In [ ]:
transform_tensor = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

In [ ]:
transform_numpy = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=15,
    brightness_range=[0.9, 1.1],
    channel_shift_range=0.1,
    rescale=1./255,
)

In [ ]:
def process_data_with_transforms(dataset, path):
    torch_data = []
    keras_data = []
    labels = []

    for _, row in dataset.iterrows():
        image_path = ''.join([path, row["images_id"], '.jpg'])

        try:
            img = Image.open(image_path)
            torch_data.append(transform_tensor(img))
            keras_data.append(np.array(img.resize((IMG_SIZE, IMG_SIZE))))
            labels.append(row["label"])

        except Exception as e:
            print(f"Error loading or processing image {image_path}: {e}")

    torch_labels = torch.tensor(labels, dtype=torch.long).to(DEVICE)
    torch_data = torch.stack(torch_data).to(DEVICE)

    num_classes = len(set(labels))
    keras_generator = transform_numpy.flow(np.stack(keras_data), y=to_categorical(np.array(labels), num_classes=num_classes), batch_size=BATCH_SIZE)

    return keras_generator, torch_data, torch_labels

In [ ]:
keras_generator, fakeVReal_Transforms_TrainData_T, fakeVReal_Transforms_TrainLabels_T = process_data_with_transforms(df_train_RealVFake, FAKEVREAL_IMAGE_PATH)

##### 2. Keras Model

In [ ]:
keras_imageTransforms = get_keras_ref_model()
keras_imageTransforms.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )

keras_imageTransforms_results = keras_imageTransforms.fit(keras_generator, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(fakeVReal_TestData, fakeVReal_TestLabels))
keras_imageTransforms_history = pd.DataFrame(keras_imageTransforms_results.history)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.5567 - loss: 0.6829 - val_accuracy: 0.5426 - val_loss: 0.6802
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5603 - loss: 0.6717 - val_accuracy: 0.6512 - val_loss: 0.6321
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6895 - loss: 0.5947 - val_accuracy: 0.8295 - val_loss: 0.4947
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8207 - loss: 0.4334 - val_accuracy: 0.9574 - val_loss: 0.1953
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9237 - loss: 0.2072 - val_accuracy: 0.9922 - val_loss: 0.0571
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9734 - loss: 0.0820 - val_accuracy: 0.9961 - val_loss: 0.0243
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9866 - loss: 0.0371 - val_accuracy: 0.9922 - val_loss: 0.0192
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9972 - loss: 0.0191 - val_accuracy: 0.9922 - val_loss:

In [ ]:
keras_imageTransforms_results.history

{'accuracy': [0.5538312196731567,
  0.5829291939735413,
  0.7090203762054443,
  0.8583899140357971,
  0.9495635032653809,
  0.9776915907859802,
  0.9864209294319153,
  0.9941803812980652,
  0.9854510426521301,
  0.9912706017494202],
 'loss': [0.6855069398880005,
  0.6619752049446106,
  0.5669069290161133,
  0.3630239963531494,
  0.15871261060237885,
  0.06568874418735504,
  0.03965779393911362,
  0.023596998304128647,
  0.04500401392579079,
  0.02375095523893833],
 'val_accuracy': [0.5426356792449951,
  0.6511628031730652,
  0.8294573426246643,
  0.9573643207550049,
  0.9922480583190918,
  0.9961240291595459,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918,
  0.9922480583190918],
 'val_loss': [0.680241584777832,
  0.6321454048156738,
  0.4946628212928772,
  0.19531673192977905,
  0.05714225023984909,
  0.0243265051394701,
  0.019230660051107407,
  0.03346909210085869,
  0.018025992438197136,
  0.021723531186580658]}

In [ ]:
keras_imageTransforms_history.to_csv(''.join(['/content/', project_folder, "/History/keras_imageTransforms_history.csv"]))
keras_imageTransforms_history

,accuracy,loss,val_accuracy,val_loss
0,0.553831,0.685507,0.542636,0.680242
1,0.582929,0.661975,0.651163,0.632145
2,0.709020,0.566907,0.829457,0.494663
3,0.858390,0.363024,0.957364,0.195317
4,0.949564,0.158713,0.992248,0.057142
5,0.977692,0.065689,0.996124,0.024327
6,0.986421,0.039658,0.992248,0.019231
7,0.994180,0.023597,0.992248,0.033469
8,0.985451,0.045004,0.992248,0.018026
9,0.991271,0.023751,0.992248,0.021724


In [ ]:
keras_imageTransforms.save(''.join(['/content/', project_folder, "/Models/keras_imageTransforms.keras"]))

##### 4. PyTorch Model

In [ ]:
train_Transforms_dataset = torch.utils.data.TensorDataset(fakeVReal_Transforms_TrainData_T, fakeVReal_Transforms_TrainLabels_T)

train_loader = DataLoader(train_Transforms_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=GENERATOR)

In [ ]:
pyTorch_imageTransforms = get_pytorch_ref_model()
pyTorch_imageTransforms_history = train_torch(pyTorch_imageTransforms, learning_rate=LEARNING_RATE)

Epoch 1/10 — accuracy: 0.5606 - loss: 0.6823 - val_accuracy 0.5388 - val_loss: 0.6961
Epoch 2/10 — accuracy: 0.7255 - loss: 0.5235 - val_accuracy 0.7791 - val_loss: 0.5835
Epoch 3/10 — accuracy: 0.9224 - loss: 0.1884 - val_accuracy 0.9457 - val_loss: 0.5195
Epoch 4/10 — accuracy: 0.9690 - loss: 0.0847 - val_accuracy 0.7209 - val_loss: 0.5188
Epoch 5/10 — accuracy: 0.9806 - loss: 0.0459 - val_accuracy 0.9729 - val_loss: 0.3389
Epoch 6/10 — accuracy: 0.9961 - loss: 0.0185 - val_accuracy 0.9729 - val_loss: 0.3077
Epoch 7/10 — accuracy: 1.0000 - loss: 0.0083 - val_accuracy 0.9806 - val_loss: 0.2164
Epoch 8/10 — accuracy: 1.0000 - loss: 0.0041 - val_accuracy 0.9884 - val_loss: 0.1865
Epoch 9/10 — accuracy: 1.0000 - loss: 0.0017 - val_accuracy 0.9845 - val_loss: 0.1618
Epoch 10/10 — accuracy: 1.0000 - loss: 0.0009 - val_accuracy 0.9806 - val_loss: 0.1505


In [ ]:
pyTorch_imageTransforms_history.to_csv(''.join(['/content/', project_folder, "/History/pyTorch_imageTransforms_history.csv"]))
pyTorch_imageTransforms_history

,accuracy,loss,val_accuracy,val_loss
0,0.560621,0.682328,0.538760,0.696114
1,0.725509,0.523459,0.779070,0.583484
2,0.922405,0.188413,0.945736,0.519509
3,0.968962,0.084682,0.720930,0.518784
4,0.980601,0.045881,0.972868,0.338890
5,0.996120,0.018526,0.972868,0.307655
6,1.000000,0.008335,0.980620,0.216450
7,1.000000,0.004119,0.988372,0.186519
8,1.000000,0.001739,0.984496,0.161764
9,1.000000,0.000899,0.980620,0.150488


In [ ]:
torch.save(pyTorch_imageTransforms.state_dict(), ''.join(['/content/', project_folder, "/Models/pyTorch_imageTransforms.pth"]))

## Part 7: Test Models

##### STEP 1: Setup

1. Reset Parameters to Baseline if needed.

In [ ]:
IMG_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.0001 # Pg. 8 of "Classification of Real and Fake Human Faces Using Deep Learning" by Fatima Maher Salman and Samy S. Abu-Naser
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2. Process the dataset.

In [21]:
testData, testLabels, testData_T, testLabels_T = basic_process_data(df_TestDataset, TEST_IMAGE_PATH, extension='')

3. Load the Test Dataset for PyTorch

In [23]:
test_dataset_T = torch.utils.data.TensorDataset(testData_T, testLabels_T)

test_loader = DataLoader(test_dataset_T, batch_size=BATCH_SIZE, shuffle=True, generator=GENERATOR)

##### STEP 2: Evaluate the Keras Models

In [26]:
keras_reference = load_model(''.join([project_folder, "/Models/model_reference.keras"]))
loss, accuracy = keras_reference.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.3790 - loss: 2.5800
Test Loss: 1.9385336637496948
Test Accuracy: 0.4922480583190918


In [27]:
keras_10xLR = load_model(''.join([project_folder, "/Models/keras_10x_LR.keras"]))
loss, accuracy = keras_10xLR.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.2648 - loss: 5.7939
Test Loss: 4.066976070404053
Test Accuracy: 0.45736435055732727


In [28]:
keras_100xLR = load_model(''.join([project_folder, "/Models/keras_100x_LR.keras"]))
loss, accuracy = keras_100xLR.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.7558 - loss: 0.6544
Test Loss: 0.696536660194397
Test Accuracy: 0.5


In [29]:
keras_additional_conv = load_model(''.join([project_folder, "/Models/keras_additional_conv.keras"]))
loss, accuracy = keras_additional_conv.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.3334 - loss: 3.3966
Test Loss: 2.3863296508789062
Test Accuracy: 0.5077519416809082


In [30]:
keras_imageTransforms = load_model(''.join([project_folder, "/Models/keras_imageTransforms.keras"]))
loss, accuracy = keras_imageTransforms.evaluate(testData, testLabels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.5848 - loss: 1.4341
Test Loss: 1.568503737449646
Test Accuracy: 0.5310077667236328


##### STEP 3: Evaluate the PyTorch Models

In [36]:
pyTorch_conversion = get_pytorch_ref_model()
pyTorch_conversion.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_conversion.pth"])))
pyTorch_conversion.to(DEVICE)
test_pytorch(pyTorch_conversion)

Test accuracy: 0.5039 - Test loss: 2.2641


In [37]:
pyTorch_10xLR = get_pytorch_ref_model()
pyTorch_10xLR.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_10X_LR.pth"])))
pyTorch_10xLR.to(DEVICE)
test_pytorch(pyTorch_10xLR)

Test accuracy: 0.4225 - Test loss: 1.9317


In [38]:
pyTorch_100xLR = get_pytorch_ref_model()
pyTorch_100xLR.load_state_dict(torch.load(''.join([project_folder, "/Models/pytorch_100X_LR.pth"])))
pyTorch_100xLR.to(DEVICE)
test_pytorch(pyTorch_100xLR)

Test accuracy: 0.5000 - Test loss: 0.6977


In [42]:
pyTorch_additional_layers = get_pytorch_ref_model_more_layers()
pyTorch_additional_layers.load_state_dict(torch.load(''.join([project_folder, "/Models/pyTorch_additional_conv.pth"])))
pyTorch_additional_layers.to(DEVICE)
test_pytorch(pyTorch_additional_layers)

Test accuracy: 0.4690 - Test loss: 3.3831


In [43]:
pyTorch_imageTransforms = get_pytorch_ref_model()
pyTorch_imageTransforms.load_state_dict(torch.load(''.join([project_folder, "/Models/pyTorch_imageTransforms.pth"])))
pyTorch_imageTransforms.to(DEVICE)
test_pytorch(pyTorch_imageTransforms)

Test accuracy: 0.5543 - Test loss: 0.9896
